In [2]:
# merge
# apply
# split

In [38]:
import pandas as pd
import os
import pickle
import numpy as np
from collections import defaultdict
import bz2
import csv
from concurrent import futures
from glob import glob

In [10]:
def set_bin_id(x, key, save_suffix=False):
    for suffix in [".fna.gz", ".fa.gz", ".fna", ".fa"]:
        if suffix in x[key]:
            if save_suffix:
                return os.path.basename(x[key]).split(suffix)[0] + suffix
            else:
                return os.path.basename(x[key]).split(suffix)[0]
    return os.path.basename(x[key])

def get_what(x, key, sep):
    lineages = x[key].split(";")
    for tax in lineages:
        if sep in tax:
            tax_ = tax.split(sep)[1]
            if tax_ == "":
                return "unclassified"
            else:
                return tax_
    return "unclassified"

In [4]:
mgs_info = pd.read_csv("/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/igo/assay/taxonomy/lineages_gtdb/oral_mgs_gtdb_taxonomy.tsv", sep='\t')

In [5]:
mgs_info.head()

,mgs_id,size,mtype,representative,MAG,oral_genome,rep_path,genome_lst,mgs_id_old,bin_id,...,lineages_strain,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new
0,mgs_1,1289,kMGS,GCF_002013135.1_ASM201313v1,0,12,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_000349405.1_ASM34940v1,GCF_004013755.1_ASM...",mgs_d0.05_1,GCF_002013135.1_ASM201313v1_genomic,...,k__Bacteria|p__Campylobacterota|c__Campylobact...,d__Bacteria;p__Campylobacterota;c__Campylobact...,k__Bacteria,k__Bacteria|p__Campylobacterota,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...,k__Bacteria|p__Campylobacterota|c__Campylobact...
1,mgs_2,1576,kMGS,YS000214_saliva.17,1572,1574,/hwfssz1/ST_META/P18Z10200N0127_MA/zhujie/yunn...,"RSZAXPI002486-19_RAH_saliva.49,RDPYD18300072_A...",mgs_d0.05_5,YS000214_saliva.spades.bin.17,...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,k__Bacteria,k__Bacteria|p__Bacteroidota,k__Bacteria|p__Bacteroidota|c__Bacteroidia,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...
2,mgs_3,6,kMGS,1150461.5.patric.fna,0,2,/hwfssz1/ST_META/P18Z10200N0127_MA/database/IG...,"SEQF2707,1150461.5.patric.fna,GCF_001005065.1_...",mgs_d0.05_9,1150461.5.patric,...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,d__Bacteria;p__Actinobacteriota;c__Actinobacte...,k__Bacteria,k__Bacteria|p__Actinobacteriota,k__Bacteria|p__Actinobacteriota|c__Actinobacteria,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,k__Bacteria|p__Actinobacteriota|c__Actinobacte...
3,mgs_4,8301,kMGS,GCF_900050145.1_6938_4_11,0,52,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_001141485.1_6925_1_59,GCF_900022795.1_1186...",mgs_d0.05_13,GCF_900050145.1_6938_4_11_genomic,...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...
4,mgs_5,1590,kMGS,GCF_001544055.1_ASM154405v1,0,1,/hwfssz1/pub/database/ftp.ncbi.nih.gov/genomes...,"GCF_000391785.1_Ente_faec_7230532-1_V1,GCF_002...",mgs_d0.05_14,GCF_001544055.1_ASM154405v1_genomic,...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,k__Bacteria,k__Bacteria|p__Firmicutes,k__Bacteria|p__Firmicutes|c__Bacilli,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...


In [6]:
# NAME LENGTH OFFSET LINEBASES LINEWIDTH QUALOFFSET
mp2_db_index = pd.read_csv("oral_mgs_rep_metaphlan2_db/mpa_v20_m200.fa.fai", sep='\t', 
                           names=["contig_name", "contig_length", "byte_index", "bases", "bytes"])

In [7]:
mp2_db_index["bin_id"] = mp2_db_index.apply(lambda x: set_bin_id(x, "contig_name", False), axis=1)
mp2_db_info = mp2_db_index.merge(mgs_info)

In [8]:
mp2_db_info.head()

,contig_name,contig_length,byte_index,bases,bytes,bin_id,mgs_id,size,mtype,representative,...,lineages_strain,classification_new,lineages_superkingdom_new,lineages_phylum_new,lineages_class_new,lineages_order_new,lineages_family_new,lineages_genus_new,lineages_species_new,lineages_strain_new
0,RSZAXPI002604-26_RA_dental.metaspades.bin.30.f...,364748,82,60,61,RSZAXPI002604-26_RA_dental.metaspades.bin.30,mgs_957,9,uMGS,RSZAXPI002604-26_RA_dental.metaspades.bin.30,...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,d__Bacteria;p__Patescibacteria;c__Saccharimona...,k__Bacteria,k__Bacteria|p__Patescibacteria,k__Bacteria|p__Patescibacteria|c__Saccharimonadia,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...
1,RSZAXPI002604-26_RA_dental.metaspades.bin.30.f...,181951,370993,60,61,RSZAXPI002604-26_RA_dental.metaspades.bin.30,mgs_957,9,uMGS,RSZAXPI002604-26_RA_dental.metaspades.bin.30,...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,d__Bacteria;p__Patescibacteria;c__Saccharimona...,k__Bacteria,k__Bacteria|p__Patescibacteria,k__Bacteria|p__Patescibacteria|c__Saccharimonadia,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...
2,RSZAXPI002604-26_RA_dental.metaspades.bin.30.f...,65726,556060,60,61,RSZAXPI002604-26_RA_dental.metaspades.bin.30,mgs_957,9,uMGS,RSZAXPI002604-26_RA_dental.metaspades.bin.30,...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,d__Bacteria;p__Patescibacteria;c__Saccharimona...,k__Bacteria,k__Bacteria|p__Patescibacteria,k__Bacteria|p__Patescibacteria|c__Saccharimonadia,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...
3,RSZAXPI002604-26_RA_dental.metaspades.bin.30.f...,62912,622965,60,61,RSZAXPI002604-26_RA_dental.metaspades.bin.30,mgs_957,9,uMGS,RSZAXPI002604-26_RA_dental.metaspades.bin.30,...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,d__Bacteria;p__Patescibacteria;c__Saccharimona...,k__Bacteria,k__Bacteria|p__Patescibacteria,k__Bacteria|p__Patescibacteria|c__Saccharimonadia,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...
4,RSZAXPI002604-26_RA_dental.metaspades.bin.30.f...,28724,687009,60,61,RSZAXPI002604-26_RA_dental.metaspades.bin.30,mgs_957,9,uMGS,RSZAXPI002604-26_RA_dental.metaspades.bin.30,...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,d__Bacteria;p__Patescibacteria;c__Saccharimona...,k__Bacteria,k__Bacteria|p__Patescibacteria,k__Bacteria|p__Patescibacteria|c__Saccharimonadia,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...,k__Bacteria|p__Patescibacteria|c__Saccharimona...


In [11]:
mp2_db_info_ = mp2_db_info.loc[:, ["mgs_id", "mgs_id_old", "bin_id", "contig_name", "contig_length", "classification_new"]]
mp2_db_info_["species"] = mp2_db_info_.apply(lambda x: get_what(x, "classification_new", "s__"), axis=1)
mp2_db_info_["genus"] = mp2_db_info_.apply(lambda x: get_what(x, "classification_new", "g__"), axis=1)
mp2_db_info_["phylum"] = mp2_db_info_.apply(lambda x: get_what(x, "classification_new", "p__"), axis=1)

In [12]:
mp2_db_info_ = mp2_db_info_.set_index("mgs_id", drop=False)

contig_id_dict = {}
contig_id_dict2 = {}

for i in mp2_db_info_.index.unique():
    contig_name_ = mp2_db_info_.loc[i, "contig_name"]
    mgs_id_old_ = mp2_db_info_.loc[i, "mgs_id_old"]
    if isinstance(contig_name_, str):
        contig_id_dict[contig_name_] = i + "_1"
        contig_id_dict2[contig_name_] = mgs_id_old_ + "_1"
    else:
        contig_name_list = contig_name_.to_list()
        mgs_id_old_list = mgs_id_old_.to_list()
        for j in range(1, len(contig_name_list) + 1):
            contig_id_dict[contig_name_list[j - 1]] = i + "_" + str(j)
            contig_id_dict2[contig_name_list[j - 1]] = mgs_id_old_list[0] + "_" + str(j)
            
another_info = pd.DataFrame(contig_id_dict.items(), columns=["contig_name", "contig_id"])
another_info2 = pd.DataFrame(contig_id_dict2.items(), columns=["contig_name", "contig_id_old"])

mp2_db_info_2 = mp2_db_info_.merge(another_info).merge(another_info2)

mp2_db_info_2.loc[:, ["contig_id", "contig_name", "contig_length",
                      "species", "genus", "phylum"]]\
             .to_csv("mp2_oral_db_metadata_v2.tsv", sep='\t', index=False, header=None)

mp2_db_info_2.loc[:, ["contig_id", "contig_name", "contig_length",
                      "species", "genus", "phylum"]]\
             .to_csv("mp2_oral_db_metadata_v2_with_header.tsv", sep='\t', index=False)

mp2_db_info_2.loc[:, ["mgs_id", "mgs_id_old", "contig_id",
                      "contig_id_old", "contig_name", "contig_length",
                      "species", "genus", "phylum"]]\
             .to_csv("mp2_oral_db_metadata_metabat2_v2.tsv", sep='\t', index=False)

In [159]:
##############

In [12]:
def get_mgs_id(row):
    return "_".join(row["ID"].split("_")[0:-1])

def get_abun_df_t2d(abun_file):
    sample_id = os.path.basename(abun_file).split(".")[0]
    abun = pd.read_csv(abun_file, sep='\t')
    abun["mgs_id"] = abun.apply(get_mgs_id, axis=1)
    
    count_df = abun.loc[:, ["mgs_id", "reads_pairs"]]\
                   .groupby("mgs_id")\
                   .agg({"reads_pairs": 'sum'})\
                   .rename(columns={"reads_pairs": sample_id})
    
    abun_df = abun.loc[:, ["mgs_id", "gene_abundance"]]\
                  .groupby("mgs_id")\
                  .agg({"gene_abundance": 'sum'})\
                  .rename(columns={"gene_abundance": sample_id})
    return count_df, abun_df

def get_abun_df_jgi(depth_file):
    sample_id = os.path.basename(depth_file).split(".")[0]
    depth = pd.read_csv(depth_file, sep='\t')\
              .rename(columns={"contigName": "contig_name"})\
              .merge(mp2_db_info_mini)\
              .groupby("mgs_id")\
              .agg({"totalAvgDepth": "mean"})
    depth[sample_id] = depth["totalAvgDepth"] / sum(depth["totalAvgDepth"])
    depth_df = depth.loc[:, ["totalAvgDepth"]].rename(columns={"totalAvgDepth": sample_id})
    abun_df = depth.loc[:, [sample_id]]
    return depth_df, abun_df

def get_all_abun_df(abun_files, func):
    count_list = []
    abun_list = []
    with futures.ProcessPoolExecutor() as pool:
        for count_df, abun_df in pool.map(func, abun_files):
            count_list.append(count_df)
            abun_list.append(abun_df)
    
    count_df_ = pd.concat(count_list, axis=1)
    abun_df_ = pd.concat(abun_list, axis=1)
    
    return count_df_, abun_df_

def get_profile(abun_tax_df, samples_list, level, out_dir, prefix, suffix=""):
    level_ = "lineages_" + level + "_new"
    #abun_tax_df[level_] = abun_tax_df.apply(lambda x: set_lineages_to(x, level), axis=1)
    _profile = abun_tax_df.loc[:, [level_] + samples_list]\
                          .melt(id_vars=[level_])\
                          .groupby(["variable", level_])\
                          .agg({"value": 'sum'})\
                          .reset_index()\
                          .pivot(index=level_,
                                 columns="variable",
                                 values="value")
    profile_ = _profile.reset_index().loc[:, [level_] + samples_list]
    profile_.to_csv(os.path.join(out_dir, "abundance_profile_%s_%s%s.tsv" % (level, prefix, suffix)),
                    sep='\t', index=False)

In [37]:
##

In [38]:
abun_df = pd.read_csv("abundance_profile_2675S.tsv", sep='\t')
count_df = pd.read_csv("count_profile_2675S.tsv", sep='\t')

samples_list = abun_df.columns[1:].to_list()
abun_tax = abun_df.merge(mgs_info)

get_profile(abun_tax, samples_list, "genus", "profile")
get_profile(abun_tax, samples_list, "strain", "profile")
get_profile(abun_tax, samples_list, "species", "profile")
get_profile(abun_tax, samples_list, "family", "profile")
get_profile(abun_tax, samples_list, "order", "profile",)
get_profile(abun_tax, samples_list, "class", "profile")
get_profile(abun_tax, samples_list, "phylum", "profile")
get_profile(abun_tax, samples_list, "superkingdom", "profile")

abun_df_jgi = pd.read_csv("abundance_profile_2675S.jgi.tsv", sep='\t')
abun_tax_jgi = abun_df_jgi.merge(mgs_info)

get_profile(abun_tax_jgi, samples_list, "genus", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "strain", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "species", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "family", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "order", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "class", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "phylum", "profile", ".jgi")
get_profile(abun_tax_jgi, samples_list, "superkingdom", "profile", ".jgi")

In [57]:
##

In [13]:
abun_df_ra = pd.read_csv("abundance_profile_ra.tsv", sep='\t')
samples_list_ra = abun_df_ra.columns[1:].to_list()
abun_tax_ra = abun_df_ra.merge(mgs_info)

get_profile(abun_tax_ra, samples_list_ra, "genus", "profile", "ra", "")
get_profile(abun_tax_ra, samples_list_ra, "strain", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "species", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "family", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "order", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "class", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "phylum", "profile", "ra")
get_profile(abun_tax_ra, samples_list_ra, "superkingdom", "profile", "ra")

abun_df_ra_jgi = pd.read_csv("abundance_profile_ra.jgi.tsv", sep='\t')
samples_list_ra = abun_df_ra_jgi.columns[1:].to_list()
abun_tax_ra_jgi = abun_df_ra_jgi.merge(mgs_info)

get_profile(abun_tax_ra_jgi, samples_list_ra, "genus", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "strain", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "species", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "family", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "order", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "class", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "phylum", "profile", "ra", ".jgi")
get_profile(abun_tax_ra_jgi, samples_list_ra, "superkingdom", "profile", "ra", ".jgi")

In [23]:
##

In [22]:
abun_df_yn = pd.read_csv("abundance_profile_yunnan.tsv", sep='\t')
samples_list_yn = abun_df_yn.columns[1:].to_list()
abun_tax_yn = abun_df_yn.merge(mgs_info)

get_profile(abun_tax_yn, samples_list_yn, "genus", "profile", "yunnan", "")
get_profile(abun_tax_yn, samples_list_yn, "strain", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "species", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "family", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "order", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "class", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "phylum", "profile", "yunnan")
get_profile(abun_tax_yn, samples_list_yn, "superkingdom", "profile", "yunnan")

abun_df_yn_jgi = pd.read_csv("abundance_profile_yunnan.jgi.tsv", sep='\t')
samples_list_yn = abun_df_yn_jgi.columns[1:].to_list()
abun_tax_yn_jgi = abun_df_yn_jgi.merge(mgs_info)

get_profile(abun_tax_yn_jgi, samples_list_yn, "genus", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "strain", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "species", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "family", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "order", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "class", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "phylum", "profile", "yunnan", ".jgi")
get_profile(abun_tax_yn_jgi, samples_list_yn, "superkingdom", "profile", "yunnan", ".jgi")

In [24]:
##

In [25]:
abun_df_pub = pd.read_csv("abundance_profile_pub.tsv", sep='\t')
samples_list_pub = abun_df_pub.columns[1:].to_list()
abun_tax_pub = abun_df_pub.merge(mgs_info)

get_profile(abun_tax_pub, samples_list_pub, "genus", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "strain", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "species", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "family", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "order", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "class", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "phylum", "profile", "pub")
get_profile(abun_tax_pub, samples_list_pub, "superkingdom", "profile", "pub")

abun_df_pub_jgi = pd.read_csv("abundance_profile_pub.jgi.tsv", sep='\t')
samples_list_pub = abun_df_pub_jgi.columns[1:].to_list()
abun_tax_pub_jgi = abun_df_pub_jgi.merge(mgs_info)

get_profile(abun_tax_pub_jgi, samples_list_pub, "genus", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "strain", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "species", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "family", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "order", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "class", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "phylum", "profile", "pub", ".jgi")
get_profile(abun_tax_pub_jgi, samples_list_pub, "superkingdom", "profile", "pub", ".jgi")

In [14]:
######

In [35]:
metadata_pub = pd.read_excel("TabS1-StudysAndSamples.xlsx", sheet_name=2)\
                 .rename(columns={"Run_id": "sample_id"})

strain_profile_pub = pd.read_csv("profile/abundance_profile_strain_pub.tsv", sep='\t')
strain_profile_pub_jgi = pd.read_csv("profile/abundance_profile_strain_pub.jgi.tsv", sep='\t')

strain_profile_pub_l = strain_profile_pub.melt(id_vars=["lineages_strain_new"],
                                               var_name="sample_id", value_name="abundance")
strain_profile_pub_jgi_l = strain_profile_pub_jgi.melt(id_vars=["lineages_strain_new"],
                                                       var_name="sample_id", value_name="abundance")

strain_profile_pub_l = strain_profile_pub_l.merge(metadata_pub)
strain_profile_pub_jgi_l = strain_profile_pub_jgi_l.merge(metadata_pub)
strain_profile_pub_l.to_csv("profile/abundance_profile_strain_pub.add_phenotype.tsv",
                            sep='\t', index=False)
strain_profile_pub_jgi_l.to_csv("profile/abundance_profile_strain_pub.add_phenotype.jgi.tsv",
                                 sep='\t', index=False)

In [36]:
##

In [126]:
def merge_profile(pattern, change=True):
    lineages = ["superkingdom", "phylum", "class", "order", "family", "genus", "species", "strain"]
    
    profile_group = {}
    profile_group_jgi = {}
    profile_dict = {}
    profile_jgi_dict = {}
    profile_0_1_dict = {}
    profile_jgi_0_1_dict = {}
    
    for i in lineages:
        profile_group[i] = []
        profile_group_jgi[i] = []
        profile_dict[i] = pd.DataFrame()
        profile_jgi_dict[i] = pd.DataFrame()
        profile_0_1_dict[i] = pd.DataFrame()
        profile_jgi_0_1_dict[i] = pd.DataFrame()
        
    for prof in glob(pattern):
        if "add_phenotype" not in prof:
            for i in lineages:
                if i in prof:
                    prof_df = pd.read_csv(prof, sep='\t').set_index("lineages_%s_new" % i)
                    if "jgi" in prof:
                        profile_group_jgi[i].append(prof_df)
                    else:
                        profile_group[i].append(prof_df)
                    break
                    
    for i in lineages:
        df = pd.concat(profile_group[i], axis=1)
        samples_list = sorted(df.columns.to_list())
        df = df.reset_index().loc[:, ["lineages_%s_new" % i] + samples_list]
        profile_dict[i] = df
        df.to_csv("profile_merged/abundance_profile_%s.tsv" % i, sep='\t', index=False)
        
        if change:
            df_ = df.set_index("lineages_%s_new" % i)
            df_[df_>=1e-6] = 1
            df_[df_<1e-6]  = 0
            df_[samples_list] = df_[samples_list].astype('int')
            df_ = df_.reset_index().loc[:, ["lineages_%s_new" % i] + samples_list]
            profile_0_1_dict[i] = df_
            df_.to_csv("profile_merged/abundance_profile_%s_0_1.tsv" % i, sep='\t', index=False)

        ##
        df_jgi = pd.concat(profile_group_jgi[i], axis=1)
        samples_list = sorted(df_jgi.columns.to_list())
        df_jgi = df_jgi.reset_index().loc[:, ["lineages_%s_new" % i] + samples_list]
        profile_jgi_dict[i] = df_jgi
        df_jgi.to_csv("profile_merged/abundance_profile_%s.jgi.tsv" % i, sep='\t', index=False)
        
        if change:
            df_jgi_ = df_jgi.set_index("lineages_%s_new" % i)
            df_jgi_[df_jgi_ >= 1e-6] = 1
            df_jgi_[df_jgi_ < 1e-6]  = 0
            df_jgi_[samples_list] = df_jgi_[samples_list].astype('int')
            df_jgi_ = df_jgi_.reset_index().loc[:, ["lineages_%s_new" % i] + samples_list]
            profile_jgi_0_1_dict[i] = df_jgi_
            df_jgi_.to_csv("profile_merged/abundance_profile_%s_0_1.jgi.tsv" % i, sep='\t', index=False)
        

    return profile_dict, profile_jgi_dict, profile_0_1_dict, profile_jgi_0_1_dict

In [127]:
profile_merged, profile_merged_jgi, profile_0_1_merged, profile_jgi_0_1_merged = merge_profile("profile/*profile*")

In [128]:
##

In [129]:
!ls -l profile_merged

total 1428180
-rw-r--r-- 1 zhujie ST_META    327513 Aug  7 16:24 abundance_profile_class_0_1.jgi.tsv
-rw-r--r-- 1 zhujie ST_META    327513 Aug  7 16:24 abundance_profile_class_0_1.tsv
-rw-r--r-- 1 zhujie ST_META   2042066 Aug  7 16:24 abundance_profile_class.jgi.tsv
-rw-r--r-- 1 zhujie ST_META   1887858 Aug  7 16:24 abundance_profile_class.tsv
-rw-r--r-- 1 zhujie ST_META   1175041 Aug  7 16:25 abundance_profile_family_0_1.jgi.tsv
-rw-r--r-- 1 zhujie ST_META   1175041 Aug  7 16:25 abundance_profile_family_0_1.tsv
-rw-r--r-- 1 zhujie ST_META   9538255 Aug  7 16:25 abundance_profile_family.jgi.tsv
-rw-r--r-- 1 zhujie ST_META   8005085 Aug  7 16:24 abundance_profile_family.tsv
-rw-r--r-- 1 zhujie ST_META   5508006 Aug  7 16:26 abundance_profile_genus_0_1.jgi.tsv
-rw-r--r-- 1 zhujie ST_META   5508006 Aug  7 16:25 abundance_profile_genus_0_1.tsv
-rw-r--r-- 1 zhujie ST_META  52639928 Aug  7 16:25 abundance_profile_genus.jgi.tsv
-rw-r--r-- 1 zhujie ST_META  40492106 Aug  7 16:25 abundance_prof

In [131]:
profile_jgi_0_1_merged["strain"].head()

,lineages_strain_new,ERR2764790,ERR2764791,ERR2764792,ERR2764793,ERR2764794,ERR2764795,ERR2764796,ERR2764797,ERR2764798,...,YS001154_saliva,YS001155_saliva,YS001156_saliva,YS001157_saliva,YS001158_saliva,YS001159_saliva,YS001160_saliva,YS001161_saliva,YS001162_saliva,YS001163_saliva
0,k__Archaea|p__Euryarchaeota|c__Methanobacteria...,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,1
1,k__Archaea|p__Nanohaloarchaeota|c__Nanosalinia...,0,1,0,0,1,1,1,1,1,...,0,1,1,1,1,1,1,1,1,1
2,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [133]:
profile_merged_jgi["strain"].head()

,lineages_strain_new,ERR2764790,ERR2764791,ERR2764792,ERR2764793,ERR2764794,ERR2764795,ERR2764796,ERR2764797,ERR2764798,...,YS001154_saliva,YS001155_saliva,YS001156_saliva,YS001157_saliva,YS001158_saliva,YS001159_saliva,YS001160_saliva,YS001161_saliva,YS001162_saliva,YS001163_saliva
0,k__Archaea|p__Euryarchaeota|c__Methanobacteria...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000003,1.172818e-08,2.811624e-07,1.226828e-07,0.000000,0.000000,0.000000,0.000006,0.000008,0.000002
1,k__Archaea|p__Nanohaloarchaeota|c__Nanosalinia...,2.417552e-07,0.000014,5.935911e-07,1.571691e-07,0.000005,0.000004,0.000003,0.000031,0.000014,...,0.000000,1.045241e-05,4.498586e-06,2.720086e-06,0.000017,0.000004,0.000005,0.000017,0.000010,0.000003
2,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,1.360128e-03,0.001395,1.386382e-03,1.431159e-03,0.001202,0.001248,0.001272,0.000399,0.000408,...,0.000014,6.097903e-06,1.802874e-05,1.571704e-05,0.000055,0.000030,0.000074,0.000082,0.000082,0.000035
3,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,1.193778e-04,0.000121,1.127043e-04,1.275291e-04,0.000115,0.000118,0.000117,0.000056,0.000071,...,0.000032,4.441092e-06,6.519913e-05,7.868814e-06,0.000070,0.000045,0.000123,0.000022,0.000012,0.000006
4,k__Bacteria|p__Actinobacteriota|c__Actinobacte...,2.499526e-04,0.000227,2.013572e-04,2.486725e-04,0.000241,0.000219,0.000215,0.000709,0.000715,...,0.000015,4.774431e-05,3.094720e-06,1.792059e-04,0.000064,0.000001,0.000013,0.000236,0.000080,0.000014
